## 2.1.2 tf.data

  머신러닝에서 많은 시간을 소요하는 것은 데이터를 다루는 것이다. 일반적으로 데이터 분석, 전처리, 파이프라인 만드는 과정에서 약 70~80%의 시간을 소비한다 해도 과언이 아니다. 텐서플로우에서는 Dataset API를 활용하여, 단순한 연구자 뿐만 아니라 서비스화를 위해서 데이터에 대한 고민들을 해결 해 주고 있다. 학습 속도를 올리기 위해  GPU가 병목현상 없이 효율적으로 최적화를 지원하는 것 뿐만 아니라 이미지, 텍스트를 넘어 넘파이 (Numpy) 및 판다스(Pandas)데이터 구조가 활용하도록 지원이 가능하다.
  
  tf.data를 사용하기 위한 데이터를 활용하기 위해, IMDB 영화 리뷰 데이터셋을 불러오자. 데이터셋은 2만5천개의 리뷰의 데이터가 긍/부정으로 나누어져 있고, 전처리도 되어 있다. IMDB 영화 리뷰 데이터셋을 선정한 이유는 MNIST와 처럼 데이터셋을 tensorflow-keras를 활용하여 가장 손쉽게 불러오는 자연어 데이터이기 때문이다.

In [2]:
import os
import tensorflow as tf #텐서플로우 모듈 불러오기

from tensorflow.keras.datasets import imdb #imdb 한글 데이터셋을 불러온다
from tensorflow.keras.preprocessing import sequence #전처리를 위한 processing 기능이다.

import numpy as np

/Users/sinseongjin/tf110/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
VOCAB_SIZE = 10000 #문장의 단어 사이즈
SENT_SIZE = 200 #문장 길이
BATCH_SIZE = 128

PAD_ID = 0 # 200단어 이하 문장에 대해서 0 값을 채워 넣는다.
START_ID = 1 # 시작 id 값
OOV_ID = 2 # out of vocab
INDEX_OFFSET = 2


#IMDB 데이터셋을 로드 합니다. 학습과 테스트 셋으로 나눕니다.
(train_input, train_label), (eval_input, eval_label) = imdb.load_data(num_words=VOCAB_SIZE,
                                                      start_char=START_ID,
                                                      oov_char=OOV_ID,
                                                      index_from=INDEX_OFFSET)

# 각 문장의 길이를 200으로 정하고, 200개가 안되는 문장에 대해서는 패딩(0)값으로 채워줍니다.
print("Pad sequences (samples x time)")
train_input = sequence.pad_sequences(train_input, 
                                 maxlen=SENT_SIZE,
                                 truncating='post',
                                 padding='post',
                                 value=PAD_ID)

eval_input = sequence.pad_sequences(eval_input, 
                                maxlen=SENT_SIZE,
                                truncating='post',
                                padding='post', 
                                value=PAD_ID)

train_len = np.array([min(len(x), SENT_SIZE) for x in train_input])
eval_len = np.array([min(len(x), SENT_SIZE) for x in eval_input])

print("train_input shape:", train_input.shape)
print("eval_input shape:", eval_input.shape)

# Pad sequences (samples x time)
# xTrain shape: (25000, 200)
# xTest shape: (25000, 200)

Pad sequences (samples x time)
train_input shape: (25000, 200)
eval_input shape: (25000, 200)


In [9]:
# IMDB데이터셋을 학습 데이터셋으로 로드 한다.

# Dataset에서 Iterator를 생성하여 초기화를 하거나 그 다음에 있는 tf.Tensor 객체를 생성하자.
#학습 셋을 Dataset에 입력한다. 입력 형태는 numpy 형태이다.
dataset = tf.data.Dataset.from_tensor_slices((train_input, train_len, train_label))
iterator = dataset.make_one_shot_iterator()
nextData = iterator.get_next()

# 세션을 실행 시켜서 간단한 테스트를 진행 해 보자
with tf.Session() as sess:
    sent_idx, sent_len, sent_label = sess.run(nextData)
    print("sentence index: {}".format(sent_idx)) # 입력 문장
    print("sentence lenght: {}".format(sent_len)) # 입력 문장의 길이
    print("sentence Label: {}".format(sent_label)) # 입력 문장 레이블 (긍/부정))
    
# 기본적인 기능 이외에도 shuffle, repeat, batch를 설정하여 데이터의 구조를 다양하게 변경 가능하다
dataset = dataset.shuffle(buffer_size=len(train_input)) # 데이터를 섞어준다.
dataset = dataset.repeat() # 데이터를 반복 진행한다
dataset = dataset.batch(BATCH_SIZE) # 데이터를 배치 형태로 생성한다

sentence index: [   1   13   21   15   42  529  972 1621 1384   64  457 4467   65 3940
    3  172   35  255    4   24   99   42  837  111   49  669    2    8
   34  479  283    4  149    3  171  111  166    2  335  384   38    3
  171 4535 1110   16  545   37   12  446    3  191   49   15    5  146
 2024   18   13   21    3 1919 4612  468    3   21   70   86   11   15
   42  529   37   75   14   12 1246    3   21   16  514   16   11   15
  625   17    2    4   61  385   11    7  315    7  105    4    3 2222
 5243   15  479   65 3784   32    3  129   11   15   37  618    4   24
  123   50   35  134   47   24 1414   32    5   21   11  214   27   76
   51    4   13  406   15   81    2    7    3  106  116 5951   14  255
    3    2    6 3765    4  722   35   70   42  529  475   25  399  316
   45    6    3    2 1028   12  103   87    3  380   14  296   97   31
 2070   55   25  140    5  193 7485   17    3  225   21   20  133  475
   25  479    4  143   29 5534   17   50   35   27  223   91 

In [10]:
# 기본적인 관련 코드에 대한 설명 구조

def parser(input_sent, length, label):
    features = {"input_sent": input_sent, "len": length} #문장의 입려과 길이를 딕셔너리 형태로 만들었다.
    return features, label

def train_input_fn():
    #학습을 위한 tf.dataset 함수 구조를 선언한다.
    dataset = tf.data.Dataset.from_tensor_slices((train_input, train_len, train_label))
    dataset = dataset.shuffle(buffer_size=len(train_input)) #데이터를 무작위로 섞는 기능을 한다. buffer_size는 데이터가 아주 클 때 활용한다.
    dataset = dataset.repeat() # 반복기능을 통해 학습 Epoch 까지 반복한다.
    dataset = dataset.map(parser) #향후 Estimator 등의 기능에서 조금 더 쉽게 데이터를 선택하기 위해 dict 구조로 만들어준다.
    dataset = dataset.batch(BATCH_SIZE) # 배치 사이즈를 설정한다.
    iterator = dataset.make_one_shot_iterator() # 위에서 한번 언급하였던 Simple하게 데이터를 불러오는 법이다.
    return iterator.get_next() #get_next()를 통하여 데이터값을 불러온다.

with tf.Session() as sess:
    print(sess.run(train_input_fn())) #간단하게 출력해본다.

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((eval_input, eval_len, eval_label))
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

({'input_sent': array([[   1,   85,   55, ...,  119,    3,  349],
       [   1,   12,  139, ...,    0,    0,    0],
       [   1,  102,  148, ...,   17,  293,   41],
       ...,
       [   1,  471,   30, ..., 5083, 1515,    2],
       [   1,    3,   85, ...,    0,    0,    0],
       [   1,    5, 1331, ...,    0,    0,    0]], dtype=int32), 'len': array([200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
  